In [1]:
from construct import *
import numpy as np
import subprocess
import sys

In [2]:
TMPrimaryHeader = BitStruct('transfer_frame_version_number' / BitsInteger(2),
                            'spacecraft_id' / BitsInteger(10),
                            'virtual_channel_id' / BitsInteger(3),
                            'ocf_flag' / Flag,
                            'master_channel_frame_count' / BitsInteger(8),
                            'virtual_channel_frame_count' / BitsInteger(8),
                            'first_header_pointer' / BitsInteger(8))

In [3]:
def seqnum(packet):
    return packet[1]*256 + packet[2]

def process_ssdv_frames(frames, output_path):
    x = np.frombuffer(bytes().join(ssdv_frames), dtype = 'uint8').reshape((-1,223))[:,5:]
    ids = set(x[:,0])
    for i in ids:
        l = list(x[x[:,0]==i,:])
        l.sort(key=seqnum)
        ssdv = '{}_{}.ssdv'.format(output_path, i)
        jpeg = '{}_{}.jpg'.format(output_path, i)
        np.array(l).tofile(ssdv)
        subprocess.call(['ssdv', '-d', '-D', ssdv, jpeg])

In [4]:
with open('raw_frame.csv') as f:
    frames = [bytes().fromhex(l.split(',')[-1].strip()) for l in f.readlines()[1:]]

In [5]:
ssdv_frames = [frame for frame in frames if TMPrimaryHeader.parse(frame).virtual_channel_id == 1]

In [6]:
process_ssdv_frames(ssdv_frames, 'output/img')